---

#### $Load$ $Libraries$

---

In [1]:
import json
import torch
import os
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModel
from huggingface_hub import notebook_login
import textwrap
import re
# !pip install -U transformers accelerate bitsandbytes datasets

---

#### $Load$ $Model$

---

REQUIREMENTS: pip install git+https://github.com/huggingface/transformers.git@096f25ae1f501a084d8ff2dcaf25fbc2bd60eba4


In [2]:
# Initialize the model name 
model_name = 'microsoft/bitnet-b1.58-2B-4T'

##### $Tokenizer$ $Set-up$


This code sets up a `tokenizer` for the language model.
It loads a pre-trained tokenizer.

In [9]:
# Load the tokenizer. The library will handle the chat template automatically.
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)

print(f"Tokenizer for {model_name} loaded successfully.")


Tokenizer for microsoft/bitnet-b1.58-2B-4T loaded successfully.


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
print(f"BitNet model  and tokenizer loaded successfully.")

BitNet model (loaded as Llama) and tokenizer loaded successfully.


In [12]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/bitnet-b1.58-2B-4T", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("microsoft/bitnet-b1.58-2B-4T", trust_remote_code=True)

OSError: microsoft/bitnet-b1.58-2B-4T does not appear to have a file named configuration_bitnet.py. Checkout 'https://huggingface.co/microsoft/bitnet-b1.58-2B-4T/tree/main' for available files.

---

####  $Zero$ $Shots$ $vs.$ $Few$ $Shots$

---

In [ ]:
# Create the folder paths for results and the folders if they do not exist
base_results_folder = '/home/lathanasopoulou/capstone/search-in-ai/prompt-decomposition/HotpotQA/llm_predictions/'
zero_shot_folder = os.path.join(base_results_folder, 'zero_shot') # Zero-shot predictions
few_shots_folder = os.path.join(base_results_folder, 'few_shot')  # Few-shot predictions
os.makedirs(zero_shot_folder, exist_ok=True)
os.makedirs(few_shots_folder, exist_ok=True)

##### $Zero$ $Shots$ $Function$

For each question, it constructs a prompt instructing the model to break down the complex question into smaller, step-by-step sub-questions. It then uses the tokenizer to convert these messages into input IDs, adds an attention mask, and generates a response from the model. Finally, it decodes the model's output and stores the original question and its zero-shot decomposition. The function returns a list of dictionaries containing these results.

* **`input_ids`**: This is the prompt, but translated into a numerical format (a list of numbers) that the model can understand. Each number corresponds to a word or part of a word.

* **`attention_mask`**: This is a list of 1s and 0s that has the same length as the input_ids. It tells the model which tokens are real words to pay attention to (a 1) and which ones are just padding that should be ignored (a 0). 

* **`pad_token_id`**: This specifies the ID of the special token that is used to "fill up" shorter sentences when we process multiple sentences at once (batching). It's directly related to the attention_mask.

* **`max_new_tokens`**: This sets the maximum length of the generated response. max_new_tokens=512 tells the model, "Do not write more than 512 new tokens after the prompt." This prevents it from writing forever.

* **`early_stopping`**: If set to False, the model might finish its thought in 80 words but feel compelled to keep adding filler words to get closer to the 300-word limit. With early stopping once it writes its thought is complete, it just stops.


* **`do_sample`**:
	* **do_sample=False**: This forces the model to be deterministic. Every time it generates a new word, it chooses the single word that it calculates as being the most statistically likely to come next. When we compare the models, we want to compare their "best, most probable" attempt at the problem.
	* **do_sample=True**: This tells the model to be creative and less predictable. Instead of always picking the #1 most likely word, it might pick the #2 or #3 most likely word, based on a random sample (controlled by parameters like temperature).

Since the task is analytical (decomposing a problem) and not creative, the deterministic approach is better. We choose do_sample=False for all the baseline experiments to ensure your results are stable and repeatable.

In [ ]:
def run_zero_shot_experiment(model, tokenizer, questions):
    results = []

    for item in questions:
        print(f"Processing (Zero-Shot) ID: {item['id']}")

        # Construct the raw string prompt
        prompt_text = (
            "You are a helpful assistant. Your task is to break down the following question "
            "into a few smaller questions that contribute to solving the overall problem.\n\n"
            f"Complex Question: {item['question']}\n\n"
            "Step-by-Step plan:"
        )

        # Apply chat template if needed (LLaDA-Instruct models require it)
        messages = [{"role": "user", "content": prompt_text}]
        formatted_prompt = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=False
        )

        # Tokenize into input IDs
        input_ids = tokenizer(formatted_prompt)['input_ids']
        input_ids = torch.tensor(input_ids).to(model.device).unsqueeze(0)  # Shape: (1, L)

        # Run LLaDA's generation function
        output_ids = generate(
            model=model,
            prompt=input_ids,
            steps=128,
            gen_length=128,
            block_length=32,
            temperature=0.0,
            cfg_scale=0.0,
            remasking='low_confidence'
        )

        # Decode the generated output after the prompt
        generated_text = tokenizer.batch_decode(output_ids[:, input_ids.shape[1]:], skip_special_tokens=True)[0].strip()

        # Store the result
        results.append({
            "id": item['id'],
            "question": item['question'],
            "decomposition": item.get('decomposition', ''),  # fallback in case not provided
            "zero_shot_decomposition": generated_text
        })

    return results


##### $Few$ $Shots$ $Function$

This function, performs few-shots learning.

It takes a list of questions, a list of high-quality example question/decomposition pairs (`shot_examples`), and the number of examples (`num_shots`) to use.

The function first uses a prompt that includes the specified number of examples, showing the model how to decompose complex questions into simpler sub-questions. Then, for each question, it sends the prompt to the language model, generates a decomposition, and stores the results. This approach helps the model understand the desired output format and style.

In [ ]:
from generate import generate  # LLaDA's custom generate function

def run_few_shot_experiment(model, tokenizer, data, shot_examples, num_shots=3):
    if num_shots > len(shot_examples):
        raise ValueError(f"You asked for {num_shots} shots, but only {len(shot_examples)} are available.")

    results = []

    for item in data:
        print(f"Processing ({num_shots}-Shot) ID: {item['id']}")

        # Build the chat-style prompt using few-shot examples
        messages = [
            {
                "role": "system",
                "content": "You are an expert assistant. Your task is to break down the following question into a few smaller questions that contribute to solving the overall problem. Give only the decomposition steps, not the final answer."
            }
        ]

        # Add few-shot examples
        for example in shot_examples[:num_shots]:
            decomposition_text = example['decomposition']
            if isinstance(decomposition_text, list):
                decomposition_text = "\n".join(decomposition_text)
            messages.append({"role": "user", "content": example['question']})
            messages.append({"role": "assistant", "content": decomposition_text})

        # Add the actual question to be decomposed
        messages.append({"role": "user", "content": item['question']})

        # Convert chat messages into a prompt string (not tokenized yet)
        prompt_text = tokenizer.apply_chat_template(
            messages,
            add_generation_prompt=True,
            tokenize=False
        )

        # Tokenize to tensor (batch of size 1)
        input_ids = tokenizer(prompt_text)['input_ids']
        input_ids = torch.tensor(input_ids).to(model.device).unsqueeze(0)

        # Generate output using LLaDA
        output_ids = generate(
            model=model,
            prompt=input_ids,
            steps=128,
            gen_length=128,
            block_length=32,
            temperature=0.0,
            cfg_scale=0.0,
            remasking='low_confidence'
        )

        # Decode only the generated portion
        generated_text = tokenizer.batch_decode(
            output_ids[:, input_ids.shape[1]:],
            skip_special_tokens=True
        )[0].strip()

        results.append({
            "id": item['id'],
            "question": item['question'],
            "decomposition": item.get('decomposition', ''),
            f"{num_shots}_shot_decomposition": generated_text
        })

    return results


##### $Save$ $the$ $results$

*Once the model processes the questions, its predictions for the zero-shot and few-shot experiments are saved in the corresponding folders*

In [ ]:
def save_results_to_json(results, folder, filename):

    # Make sure the folder exists
    os.makedirs(folder, exist_ok=True)

    # Construct the full path for the file
    full_path = os.path.join(folder, filename)

    # Save the file
    with open(full_path, 'w', encoding='utf-8') as f:
        json.dump(results, f, ensure_ascii=False, indent=4)

    print("Results have been saved!")


---

#### $HotpotQA$ $Dataset$ $Predictions$

---

In [ ]:
hotpot_dataset_path = "/home/lathanasopoulou/capstone/search-in-ai/prompt-decomposition/HotpotQA/HotpotQA_dataset/hotpot_dataset.json" # Define the path to the hotpot dataset

# Load the dataset
with open(hotpot_dataset_path, "r") as file:
    hotpot_data = json.load(file)
    print(f"Loaded {len(hotpot_data)} questions.")


Loaded 5 questions.


In [ ]:
# Display the first question
print("First question details:")
print("ID:", hotpot_data[0]["id"])
print("Question:", hotpot_data[0]["question"])
print("Answer:", hotpot_data[0]["answer"])
print("Supporting sentences:", hotpot_data[0]["supporting_sentences"])
print("-"*150)
print("Decomposition:", hotpot_data[0]["decomposition"])


First question details:
ID: 5a8b57f25542995d1e6f1371
Question: Were Scott Derrickson and Ed Wood of the same nationality?
Answer: yes
Supporting sentences: ['Scott Derrickson (born July 16, 1966) is an American director, screenwriter and producer.', 'Edward Davis Wood Jr. (October 10, 1924 – December 10, 1978) was an American filmmaker, actor, writer, producer, and director.', 'Aggregating the above we conclude that the answer is: yes']
------------------------------------------------------------------------------------------------------------------------------------------------------
Decomposition: ['What nationality Scott Derrickson had?', 'What nationality Ed Wood had?', 'Was the nationality the same?']


In [ ]:
# Define the name of the model for our file names
model_file_name = "LLaDA-8B-Base_Hotpot_results.json"

In [ ]:
few_shot_hotpot_examples_path = "/home/lathanasopoulou/capstone/search-in-ai/prompt-decomposition/HotpotQA/HotpotQA_dataset/hotpot_few_shot.json"
with open(few_shot_hotpot_examples_path, "r") as file:
    shot_examples = json.load(file)
    print(f"Loaded {len(hotpot_data)} questions.")

Loaded 5 questions.


##### $Experiments$ $Run$


In [ ]:
# Run the zero-shot experiment
print("\nStarting Zero-Shot Experiment")
zero_shot_results = run_zero_shot_experiment(model, tokenizer, hotpot_data)
save_results_to_json(zero_shot_results, zero_shot_folder, model_file_name)


Starting Zero-Shot Experiment
Processing (Zero-Shot) ID: 5a8b57f25542995d1e6f1371
Processing (Zero-Shot) ID: 5a8c7595554299585d9e36b6
Processing (Zero-Shot) ID: 5a85ea095542994775f606a8
Processing (Zero-Shot) ID: 5adbf0a255429947ff17385a
Processing (Zero-Shot) ID: 5a8e3ea95542995a26add48d
Results have been saved!


In [ ]:
# model_file_name = "LLaDA-8B-Base_Hotpot_results.json"  # ✅ no leading slash

# save_results_to_json(
#     zero_shot_results,
#     '/home/lathanasopoulou/capstone/LLaDA',
#     model_file_name
# )


In [ ]:
# Run the few-shot experiment with 3 shots
print("\nStarting 3-Shot Experiment")
three_shot_results = run_few_shot_experiment(model, tokenizer, hotpot_data, shot_examples, num_shots=3)
save_results_to_json(three_shot_results, few_shots_folder, f"3shot_{model_file_name}")


Starting 3-Shot Experiment
Processing (3-Shot) ID: 5a8b57f25542995d1e6f1371
Processing (3-Shot) ID: 5a8c7595554299585d9e36b6
Processing (3-Shot) ID: 5a85ea095542994775f606a8
Processing (3-Shot) ID: 5adbf0a255429947ff17385a
Processing (3-Shot) ID: 5a8e3ea95542995a26add48d
Results have been saved!
